In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model 

In [3]:
#Load  trained model,scaler pickle,one hot encoded pickle
model = load_model('model.keras')

#load the encoder and scaler
with open ('scaler.pkl','rb') as file:
    scaler = pickle.load(file)
with open ('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
with open ('OneHot_Encoder.pkl','rb') as file:
    OneHot_Encoder = pickle.load(file)

In [25]:
#Example input data 
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [27]:
#One hot encode geography
geo_encoder = OneHot_Encoder.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder,columns=OneHot_Encoder.get_feature_names_out(['Geography']))
geo_encoder_df

c:\Gen AI projects\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [33]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [35]:
#Encode categorical variables
input_df['Gender']=label_encoder_gender.transform([input_df['Gender']])

c:\Gen AI projects\ANN Classification\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [36]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [37]:
#Concat One Hot Encoded data

input_df=pd.concat([input_df.drop(['Geography'],axis=1),geo_encoder_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [42]:
#Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
#Predict the churn
prediction = model.predict(input_scaled)
prediction 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


array([[0.02505912]], dtype=float32)

In [41]:

#prediction probability
prediction_prob = prediction[0][0]

if prediction_prob > 0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn


In [43]:
import streamlit as st
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle
import pandas as pd

#Load the trained model

model = tf.keras.models.load_model('model.keras')

#load encoders and scaler

with open('OneHot_Encoder.pkl','rb') as file:
    OneHot_Encoder = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

#User input
geography = st.selectbox('Geography',OneHot_Encoder.categories_[0])
gender = st.selectbox('Gender',label_encoder_gender.classes_)
age = st.slider('Age',18,99)
balance = st.number_input('Balance')
Credit_Score = st.number_input('Credit Score')
Estimated_Salary = st.number_input('Estimated Salary')
tenure = st.slider('Tenure',0,10)
Num_of_Products = st.slider('Number of products',1,4)
Has_Cr_Card = st.selectbox('Has Credit Card',[0,1])
Is_active_member = st.selectbox('Is active member',[0,1])

user_input_df = pd.DataFrame({
    'CreditScore' : [Credit_Score],
    'Gender' : [label_encoder_gender.transform([gender])[0]],
    'Age' : [age],
    'Balance' : [balance],
    'EstimatedSalary' : [Estimated_Salary],
    'Tenure' : [tenure],
    'NumOfProducts' : [Num_of_Products],
    'HasCrCard' : [Has_Cr_Card],
    'IsActiveMember' : [Is_active_member]

})

#One Hot Encode 'Geography'

OneHot_Encoder_geo = OneHot_Encoder.transform([[geography]]).toarray()
OneHot_Encoder_geo_df = pd.DataFrame(OneHot_Encoder_geo,columns=OneHot_Encoder.get_feature_names_out(['Geography']))
 
#Concat One Hot encoder data

user_input= pd.concat([user_input_df.reset_index(drop=True),OneHot_Encoder_geo_df],axis=1)

#Scale the input data

scaled_input = scaler.transform(user_input)

#churn prediction

prediction = model.predict(scaled_input)

#prediction propability

prediction_prob = prediction[0][0]

if prediction_prob > 0.5:
    st.write("Customer is likely to churn")
else:
    st.write("Customer is unlikely to churn")

c:\Gen AI projects\ANN Classification\venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-12-01 22:02:59.450 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 22:02:59.452 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 22:02:59.470 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 22:02:59.472 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunCont

ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.
